In [28]:
import pandas as pd
import json
import xml.etree.ElementTree as ET
import logging
import os

In [29]:
# Set up logging to a file
logging.basicConfig(filename='etl_process.log', level=logging.INFO, format='%(asctime)s - %(message)s')

In [30]:
#Function to extract data from CSV files
def extract_csv(file_path):
    logging.info(f"Extracting CSV file: {file_path}")
    return pd.read_csv(file_path)

In [31]:
#Function to extract data from JSON files
def extract_json(file_path):
    logging.info(f"Extracting JSON file: {file_path}")
    with open(file_path, 'r') as file:
        data = json.load(file)
    return pd.DataFrame(data)


In [32]:
#Function to extract data from XML files
def extract_xml(file_path):
    logging.info(f"Extracting XML file: {file_path}")
    tree = ET.parse(file_path)
    root = tree.getroot()
    rows = []
    for child in root:
        row = {}
        for element in child:
            #storing the element tag and the text in dict row 
            row[element.tag] = element.text
        rows.append(row)
    return pd.DataFrame(rows)

In [33]:
#Function to extract the data from different formats and combining it to a single dataframe
def extract_data(directory):
    logging.info("Starting extraction phase")
    dfs = []
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        if file_name.endswith('.csv'):
            dfs.append(extract_csv(file_path))
        elif file_name.endswith('.json'):
            dfs.append(extract_json(file_path))
        elif file_name.endswith('.xml'):
            dfs.append(extract_xml(file_path))
    combined_df = pd.concat(dfs, ignore_index=True)
    logging.info("Extraction phase completed")
    return combined_df

In [34]:
#Function to transform the height in inches to meters and weight in pounds to kgs
def transform_data(df):
    logging.info("Starting transformation phase")
    df = df.astype({"name":'object', "height":'float64', "weight":'float64'})
    df['height'] = df['height'].apply(lambda x: x * 0.0254 if x > 0 else x)  # Assuming heights are in inches
    df['weight'] = df['weight'].apply(lambda x: x * 0.453592 if x > 0 else x)  # Assuming weights are in pounds
    logging.info("Transformation phase completed")
    return df

In [35]:
#Function to load the transformed data into csv file
def load_data(df, output_file):
    logging.info("Starting loading phase")
    df.to_csv(output_file, index=False)
    logging.info("Loading phase completed")

In [36]:
# Main function to execute ETL process
def etl_process(input_directory, output_file):
    logging.info("ETL process started")
    df = extract_data(input_directory)
    df = transform_data(df)
    load_data(df, output_file)
    logging.info("ETL process completed")

In [37]:
#Setting the directory path and output file to store the transformed data
input_directory = r"C:\Users\admin\Data Eng Workspace\Comprehensive_ETL_Workflow"
output_file = r"C:\Users\admin\Data Eng Workspace\Comprehensive_ETL_Workflow\Transformed_data\Output_transformed.csv"
etl_process(input_directory,output_file)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
